##   Import Module$

In [2]:

import os
import sys
notebook_dir = os.path.abspath(os.path.dirname(''))
parent_dir = os.path.abspath(os.path.join(notebook_dir, os.pardir))
sys.path.append(parent_dir)
from BrillMoore import BrillMoreAcc,BrillMore
from Trie import Trie
from collections import Counter
import pickle

## Utility Function$

In [9]:

# $ample True or False for a given success probability
def with_probability(probability):
    # Generate a random number between 0 and 1
    random_value = random.uniform(0, 1)
    
    # Check if the random number is less than the given probability
    return random_value < probability


# $ample a category from a given categorie$ with probability given a$ a dictionary
def sample_from_probability(probabilities):
    # Check if the probabilities sum to 1
    if abs(sum(probabilities.values()) - 1.0) > 1e-10:
        raise ValueError("Probabilities must sum to 1")

    # Sample from the given probability distribution
    outcome = random.choices(list(probabilities.keys()), weights=list(probabilities.values()), k=1)[0]
    return outcome

#Load the error model for generating error in the sentence
with open(os.path.join(notebook_dir, '..','models','bma_jan13.pickle'),'rb') as f:
    bma = pickle.load(f)

In [4]:
bma.edit_dict

defaultdict(int,
            {(None, '्'): 0.09197738042690697,
             ('द', 'ध'): 0.14152531399715138,
             ('ः', None): 0.6431564245810056,
             ('व', 'म'): 0.007462492524506617,
             ('ध', 'म'): 0.061197916666666664,
             (None, 'य'): 0.02408534807991805,
             (None, 'ा'): 0.16059397200036643,
             ('ि', 'ी'): 0.8259324271393962,
             ('अ', 'आ'): 0.036955520294825205,
             ('ी', 'ि'): 0.38148861919891663,
             (None, 'ी'): 0.08459020762369558,
             (None, 'े'): 0.04563057473370366,
             ('ै', 'ा'): 0.509728846566435,
             ('क', 'त'): 0.04701975179164482,
             ('ा', 'ो'): 0.5677122045488331,
             ('म', None): 0.16428536597259472,
             ('ी', None): 0.3848325433616334,
             (None, 'न'): 0.07391336936696843,
             ('ा', 'ै'): 0.08620619754854926,
             ('ज', 'म'): 0.16329704510108864,
             ('भ', 'ण'): 0.01187308085977482,
           

In [14]:
# Probability dict $elector

dict_selector = {}
for k1 in bma.edit_dict:
    temp_dict = {}
    if k1[0] in dict_selector:        
        continue
    for k2 in bma.edit_dict:
        if k2[0] == k1[0]:
            temp_dict[k2] = bma.edit_dict[k2]
    dict_selector[k1[0]] = temp_dict
    

In [6]:
dict_selector.keys()

dict_keys([None, 'द', 'ः', 'व', 'ध', 'ि', 'अ', 'ी', 'ै', 'क', 'ा', 'म', 'ज', 'भ', '्', 'े', 'ग', 'न', 'ल', 'ू', 'त', 'ब', 'ण', 'ऊ', 'स', 'ु', 'ए', 'य', 'र', 'ह', 'च', 'आ', 'फ', 'ङ', 'इ', 'ँ', 'ो', 'प', 'ई', 'ं', 'छ', 'श', 'ट', 'ष', 'थ', 'ख', 'उ', 'ढ', 'घ', 'झ', 'ौ', 'ृ', 'ठ', 'ड', 'ओ', 'ऐ', 'ञ', 'औ'])

## $ample Generation

In [7]:

# $ample Generation of error
text = 'नेपालीले धेरै काम गर्छन'
tokens = text.split()
orig_tokens = tokens.copy()
print("Probability of error in each character", 1-bma.error_rate)

Probability of error in each character 0.010000000000000009


In [7]:

for i in range(len(tokens)):
    
    # List that contains True if the character is to be errored 
    result = [with_probability(1-bma.error_rate) for i in range(len(tokens[i]))]
    print(result)
    
    # List with replacement character if the character is to be errored
    li = [ch if (b!= True or ch not in dict_selector) else sample_from_probability(dict_selector[ch])[1] for ch,b in zip(tokens[i],result)]
    print(li)
    
    # Create a final token by joining
    tokens[i] = ''.join(['' if ch is None else ch for ch in li])
    


[False, False, False, False, False, False, False, False]
['न', 'े', 'प', 'ा', 'ल', 'ी', 'ल', 'े']
[False, False, False, False]
['ध', 'े', 'र', 'ै']
[False, False, False]
['क', 'ा', 'म']
[False, False, False, False, False]
['ग', 'र', '्', 'छ', 'न']


In [8]:
tokens,orig_tokens

(['नेपालीले', 'धेरै', 'काम', 'गर्छन'], ['नेपालीले', 'धेरै', 'काम', 'गर्छन'])

## Passing through the noisy channel

In [10]:
import regex as re
def words(text): 
    text = re.sub(r'[\u0964]', r'\u0020\u0964\u0020', text)
    return re.findall(r'[\u0900-\u097F]+', text.lower())

In [10]:
with open(os.path.join(notebook_dir, '..','data','wikipedia.dev.ne-en.ne'),'r',encoding = 'utf-8') as f:
    sentences = f.read().split('\n')

In [5]:

## Or run this to gather sentences
import csv

def read_csv_file(file_path):
    data = []
    with open(file_path, 'r',encoding='utf-8') as csv_file:
        csv_reader = csv.reader(csv_file)
        for row in csv_reader:
            data.append(row)
    return data

# Example usage:
file_path = os.path.join(notebook_dir, '..','data','raw_sentences_np_65k.csv')
csv_data = read_csv_file(file_path)
sentences = [l[0]+' ।' for l in csv_data][1:]

In [17]:
sentences

['अब गाह्रो छैन प्रदूषण नाप्न भारतीय पूर्व प्रधानमन्त्रीलाई उपप्रधानमन्त्री निधीले किन भेटे ।',
 'लाठी लिएका हजारौँ संख्यामा रहेका थारुहरुले र्याली प्रर्दशन गरी राजापुर बजारको दक्षिण चौकमा कोणसभा गरेका थिए ।',
 'आफूहरुले आफनो स्वाभिमानको लडाइ लडेको उनको भनाई थियो ।',
 'प्रदर्शनकारीले प्रहरी दमन बन्द गर, कैलाली कञ्चनपुर छाड्दैनौँ , प्रतिगमनकारी होसियार जस्ता नारा लगाएका थिए ।',
 'भारतको अघोषित नाकाबन्दीका कारण पेट्रोलियम पदार्थको अभाव भएपछि सर्वसाधारणले जोखिमपूर्ण यात्रा गर्नुपर्ने बाध्यता बढेको छ ।',
 'पेट्रोलियम पदार्थ पाउन छाडेपछि सिन्धुपाल्चोक जिल्लाको बाह्रविसे, चौतारा, मेलम्चीका सर्वसाधारणले निकै कष्ट भोग्नु परेका छन् ।',
 'बसभित्र खोँचाखोँच यात्रु हुन्छन् कतिपय त निस्सासिएर बेहोससमेत हुने गर्छन् ।',
 'विद्यालय टाढा भएकाले बसले समेत विद्यार्थीलाई रोक्दैन, हिँडेर जाँदा विद्यालय समयमा पुग्न नसकिने विद्यार्थीले गुनासो पोखे ।',
 'प्रमुख सत्तारुढ दल नेपाली काँग्रेसको नेतृत्वमा राष्ट्रिय सहमतिको सरकार गठन हुनुपर्ने सो पार्टीका अधिकांश नेताहरुको अभिमत रहेको छ ।',
 'यद्यपि काँग्रेस नेतृत्

In [18]:
sentences

['अब गाह्रो छैन प्रदूषण नाप्न भारतीय पूर्व प्रधानमन्त्रीलाई उपप्रधानमन्त्री निधीले किन भेटे ।',
 'लाठी लिएका हजारौँ संख्यामा रहेका थारुहरुले र्याली प्रर्दशन गरी राजापुर बजारको दक्षिण चौकमा कोणसभा गरेका थिए ।',
 'आफूहरुले आफनो स्वाभिमानको लडाइ लडेको उनको भनाई थियो ।',
 'प्रदर्शनकारीले प्रहरी दमन बन्द गर, कैलाली कञ्चनपुर छाड्दैनौँ , प्रतिगमनकारी होसियार जस्ता नारा लगाएका थिए ।',
 'भारतको अघोषित नाकाबन्दीका कारण पेट्रोलियम पदार्थको अभाव भएपछि सर्वसाधारणले जोखिमपूर्ण यात्रा गर्नुपर्ने बाध्यता बढेको छ ।',
 'पेट्रोलियम पदार्थ पाउन छाडेपछि सिन्धुपाल्चोक जिल्लाको बाह्रविसे, चौतारा, मेलम्चीका सर्वसाधारणले निकै कष्ट भोग्नु परेका छन् ।',
 'बसभित्र खोँचाखोँच यात्रु हुन्छन् कतिपय त निस्सासिएर बेहोससमेत हुने गर्छन् ।',
 'विद्यालय टाढा भएकाले बसले समेत विद्यार्थीलाई रोक्दैन, हिँडेर जाँदा विद्यालय समयमा पुग्न नसकिने विद्यार्थीले गुनासो पोखे ।',
 'प्रमुख सत्तारुढ दल नेपाली काँग्रेसको नेतृत्वमा राष्ट्रिय सहमतिको सरकार गठन हुनुपर्ने सो पार्टीका अधिकांश नेताहरुको अभिमत रहेको छ ।',
 'यद्यपि काँग्रेस नेतृत्

In [11]:
sentences_tokens = [words(s) for s in sentences]
sentences_tokens

[['अब',
  'गाह्रो',
  'छैन',
  'प्रदूषण',
  'नाप्न',
  'भारतीय',
  'पूर्व',
  'प्रधानमन्त्रीलाई',
  'उपप्रधानमन्त्री',
  'निधीले',
  'किन',
  'भेटे',
  '।'],
 ['लाठी',
  'लिएका',
  'हजारौँ',
  'संख्यामा',
  'रहेका',
  'थारुहरुले',
  'र्याली',
  'प्रर्दशन',
  'गरी',
  'राजापुर',
  'बजारको',
  'दक्षिण',
  'चौकमा',
  'कोणसभा',
  'गरेका',
  'थिए',
  '।'],
 ['आफूहरुले',
  'आफनो',
  'स्वाभिमानको',
  'लडाइ',
  'लडेको',
  'उनको',
  'भनाई',
  'थियो',
  '।'],
 ['प्रदर्शनकारीले',
  'प्रहरी',
  'दमन',
  'बन्द',
  'गर',
  'कैलाली',
  'कञ्चनपुर',
  'छाड्दैनौँ',
  'प्रतिगमनकारी',
  'होसियार',
  'जस्ता',
  'नारा',
  'लगाएका',
  'थिए',
  '।'],
 ['भारतको',
  'अघोषित',
  'नाकाबन्दीका',
  'कारण',
  'पेट्रोलियम',
  'पदार्थको',
  'अभाव',
  'भएपछि',
  'सर्वसाधारणले',
  'जोखिमपूर्ण',
  'यात्रा',
  'गर्नुपर्ने',
  'बाध्यता',
  'बढेको',
  'छ',
  '।'],
 ['पेट्रोलियम',
  'पदार्थ',
  'पाउन',
  'छाडेपछि',
  'सिन्धुपाल्चोक',
  'जिल्लाको',
  'बाह्रविसे',
  'चौतारा',
  'मेलम्चीका',
  'सर्वसाधारणले',
  'निकै',
  'कष्ट'

In [12]:
#error_rate = bma.error_rate
error_rate = 0.9

def make_error(tokens):
    
    for i in range(len(tokens)):

        # List that contains True if the character is to be errored 
        result = [with_probability(1-error_rate) for i in range(len(tokens[i]))]

        # List with replacement character if the character is to be errored
        li = [ch if (b!= True or ch not in dict_selector) else sample_from_probability(dict_selector[ch])[1] for ch,b in zip(tokens[i],result)]

        # Create a final token by joining
        tokens[i] = ''.join(['' if ch is None else ch for ch in li])
        
    return tokens

In [15]:
correct_incorrect = []
for t in sentences_tokens:
    correct_incorrect.append((' '.join(t),' '.join(make_error(t.copy()))))
    
    

In [16]:
len(correct_incorrect),correct_incorrect

(65311,
 [('अब गाह्रो छैन प्रदूषण नाप्न भारतीय पूर्व प्रधानमन्त्रीलाई उपप्रधानमन्त्री निधीले किन भेटे ।',
   'अब गाह्रो छैन प्रदुषण नाप्न भाछताय पूर्व प्रढानमन्त्रीलाई उपप्रधानमन्त्रि निधीले किन भेटे ।'),
  ('लाठी लिएका हजारौँ संख्यामा रहेका थारुहरुले र्याली प्रर्दशन गरी राजापुर बजारको दक्षिण चौकमा कोणसभा गरेका थिए ।',
   'लाठी लिएका हजारौँ संख्योमा ढहेका पारुहरूले र्याखी प्रर्दशन गली राजापुर बबारको दक्षीण चोकमा कोणसभा गरेका थिए ।'),
  ('आफूहरुले आफनो स्वाभिमानको लडाइ लडेको उनको भनाई थियो ।',
   'आफूहवुले आफनो स्वाटिमाछको लडाइ लडेको उनको भनई शिो ।'),
  ('प्रदर्शनकारीले प्रहरी दमन बन्द गर कैलाली कञ्चनपुर छाड्दैनौँ प्रतिगमनकारी होसियार जस्ता नारा लगाएका थिए ।',
   'प्रदर्शनकारले प्नरी दमन खन्द गर कैदाली कञ्चनपुर छोड्दैनौँ प्रतिगमनारी होसियोर जस्ता ारा लजैएका थिए ।'),
  ('भारतको अघोषित नाकाबन्दीका कारण पेट्रोलियम पदार्थको अभाव भएपछि सर्वसाधारणले जोखिमपूर्ण यात्रा गर्नुपर्ने बाध्यता बढेको छ ।',
   'भोतका अघाषित नोकाबन्दाका कारण पेट्ोलियम पभार्थको अभाव भएपछि सरवशाधरणले जोखीमपूर्ण यात्रा गर्

## Passing through the noisy model with variable error rate

In [16]:
total = sum(bma.count_dict.values())
count_probab = {k:(bma.count_dict[k]/total) for k in bma.count_dict}
count_probab['n'] = count_probab[None]

In [17]:
sum(count_probab.values())

1.1290388794198498

In [18]:
sorted(count_probab.items(),key = lambda x:x[1],reverse = True)

[(None, 0.12903887941985),
 ('n', 0.12903887941985),
 ('ी', 0.10316296944282646),
 ('ब', 0.08272704020771227),
 ('ा', 0.07952237355967322),
 ('ि', 0.07408024347728899),
 ('ै', 0.06511749369705906),
 ('क', 0.05533369585542676),
 ('व', 0.04133074266161428),
 ('न', 0.03768223720551349),
 ('ु', 0.03504714578331639),
 ('स', 0.02218334304118781),
 ('म', 0.02203826255913373),
 ('अ', 0.020995832428819215),
 ('ल', 0.014935766959908352),
 ('ई', 0.014365117063828965),
 ('श', 0.013062616736054541),
 ('र', 0.012966971084922592),
 ('्', 0.012800397198119757),
 ('ू', 0.012546775022084473),
 ('ग', 0.011109940914630346),
 ('त', 0.010256652746104858),
 ('े', 0.009929952993923814),
 ('द', 0.008299678243730912),
 ('प', 0.008207256603311274),
 ('ो', 0.00753021435372556),
 ('ज', 0.006219116664051636),
 ('ह', 0.006117022991495059),
 ('य', 0.005854803453560275),
 ('ट', 0.005493714253781227),
 ('भ', 0.005249764109882883),
 ('ष', 0.0050595474778564195),
 ('ध', 0.004952080454112656),
 ('च', 0.004743594428049753)

In [19]:
dict_selector['n'] = dict_selector[None]
dict_selector['n']

{(None, '्'): 0.09197738042690697,
 (None, 'य'): 0.02408534807991805,
 (None, 'ा'): 0.16059397200036643,
 (None, 'ी'): 0.08459020762369558,
 (None, 'े'): 0.04563057473370366,
 (None, 'न'): 0.07391336936696843,
 (None, 'ै'): 0.036502794133568744,
 (None, 'क'): 0.024476776627551572,
 (None, 'प'): 0.006745896246450076,
 (None, 'स'): 0.027949663954427724,
 (None, 'व'): 0.009527537414739367,
 (None, 'त'): 0.02657549990422493,
 (None, 'ि'): 0.0495781732779226,
 (None, 'ं'): 0.02191167040050636,
 (None, 'ण'): 0.025767658008045105,
 (None, 'म'): 0.0249015182430688,
 (None, 'ह'): 0.0023568995527720638,
 (None, 'थ'): 0.010185470505442523,
 (None, 'ु'): 0.014541154131236831,
 (None, 'ष'): 0.0007745288282961199,
 (None, 'र'): 0.04045872094475861,
 (None, 'ल'): 0.012026017506017173,
 (None, 'ध'): 0.005321762594421727,
 (None, 'ट'): 0.006037993553921365,
 (None, 'ो'): 0.017047962489485563,
 (None, 'ँ'): 0.06488552797048462,
 (None, 'ौ'): 0.000849483231034454,
 (None, 'द'): 0.005821458612677288,
 (No

In [20]:


def make_error2(tokens):
    
    for i in range(len(tokens)):
        
        # Add None token between characters to model the error that can happen due to addition of string
        temp = ''
        for j,t in enumerate(tokens[i]):
            temp+=t

            if j == len(tokens[i])-1:
                continue
                
            # Don't add 'n' or None token when previo$ and ju$t after token are character modifier
            if t not in ['े','ै','ो','ौ','्','ॎ','ि','ी','ु','ू','ृ','ा'] and tokens[i][j+1] not in ['े','ै','ो','ौ','्','ॎ','ि','ी','ु','ू','ृ','ा']:
                temp+='n'
    
        
        # List that contains True if the character is to be errored 
        result = [with_probability(count_probab[t]) if t in dict_selector else False for t in temp]

        # List with replacement character if the character is to be errored
        li = [ch if (b!= True or ch not in dict_selector) else sample_from_probability(dict_selector[ch])[1] for ch,b in zip(temp,result)]

        # Create a final token by joining
        tokens[i] = ''.join(['' if ch is None or ch == 'n' else ch for ch in li])
        
    return tokens

In [21]:
correct_incorrect2 = []
for t in sentences_tokens:
    correct_incorrect2.append((' '.join(t),' '.join(make_error2(t.copy()))))
    
    

In [31]:
correct_incorrect2

[('अब गाह्रो छैन प्रदूषण नाप्न भारतीय पूर्व प्रधानमन्त्रीलाई उपप्रधानमन्त्री निधीले किन भेटे ।',
  'अब गाह्रो छोन प्रदुषण नाप्न भारतीय पूर्व प्रधानमन्त्रीलाई उपप्रधानमन्क्री निधोले कीन भेटे ।'),
 ('लाठी लिएका हजारौँ संख्यामा रहेका थारुहरुले र्याली प्रर्दशन गरी राजापुर बजारको दक्षिण चौकमा कोणसभा गरेका थिए ।',
  'लाठी लिएका हजारौँ संख्यामा रहेका थारुहरुले र्याली प्रर्दशन गरी राजापुर बजारको दक्षिण चौकमा कोणसभा गरेका थिए ।'),
 ('आफूहरुले आफनो स्वाभिमानको लडाइ लडेको उनको भनाई थियो ।',
  'आफूहरुले आफनो स्वाभिमानको लडाइ लडेको उनको भनाई थियो ।'),
 ('प्रदर्शनकारीले प्रहरी दमन बन्द गर कैलाली कञ्चनपुर छाड्दैनौँ प्रतिगमनकारी होसियार जस्ता नारा लगाएका थिए ।',
  'प्रदर्शनकारीले प्रहरी दमन बन्द गर कैलाली कञ्चनपुर छाड्दैनौँ प्रतीगमनकारी होसियार जस्ता नारा लगाएका थिए ।'),
 ('भारतको अघोषित नाकाबन्दीका कारण पेट्रोलियम पदार्थको अभाव भएपछि सर्वसाधारणले जोखिमपूर्ण यात्रा गर्नुपर्ने बाध्यता बढेको छ ।',
  'भारतको अघोषित नाकाबन्दीका कारण पेट्रोलियम पदेर्थको अभाव भएपछि सर्असाधारले जोखिमपूर्ण यात्रा गर्नुपर्ने ब

In [23]:
for t in 'कमिला':
    print(t)

क
म
ि
ल
ा


In [25]:
correct_incorrect2 = []
for t in sentences_tokens:
    correct_incorrect2.append((' '.join(t),' '.join(make_error2(t.copy()))))
    

In [25]:
correct_incorrect2

[('पुटिनको आफ्नै नाइट उल्भ्स नामको बाइक राइडर ग्याङ पनि छ ।',
  'पुटिनको आफ्नै नाइअट उल्भ्स नामको वाइक राइडर ग्याङ पनि छ ।'),
 ('पुटिनको तर्क छ अमेरिका र युरोपको आडमा कोसोभोले आत्मनिर्णयको अधिकार उपयोग गर्न पाउँछ भने रुसको समर्थन पाएको क्रिमियाले नपाउने कारण के छ',
  'पुटिनको तर्क छ अंमेरिका र युरोपको आडमा कोसोभोले आत्मनिर्णयको अधीकोर उपयोग गीर्न पाउँछ भने रुसको समर्थन पाएको इ्रिमियाले नपोउने करिण के छ'),
 ('पुटिनको विरोधमा गरिने सबै र् याली र सभाहरूमा यो गीत आधिकारिकजस्तै बनेको थियो ।',
  'पुटिनको विरोधमा गरिने सबै र् याली र सभाहरूमा यो गीत आधिकारिकजस्तै बनेको थियो ।'),
 ('पुटिनको शब्दमा घुँडा टेकेर बसेको रुस अब बिस्तारै आफ्नो खुट्टामा उभिएको छ ।',
  'पटिनको शब्दमा घुँडा टेकेर बसेको रुस अब बिस्तारै आफ्नो खुट्टाथा उभिएको छ ।'),
 ('पुटिन राष्ट्रपति भएको दुई वर्ष हुँदा लेखिएको गीत हो अ म्यान लाइक पुटिन ।',
  'पूटिन राष्ट्र्पति भएीको दुई वर्ष हुँयदा लेखिएको गीत हो अ म्यान लाइक पुटिन ।'),
 ('पुटिनलाई गीतमा देशलाई सही दिशामा डोहोर् याउने असल नेता उदाहरणीय व्यक्तित्व र अनुकरणीय पतिका रूपमा द

In [32]:
with open(os.path.join(notebook_dir,'..','data','eval_data2.pic'),'wb') as f:
    pickle.dump(correct_incorrect2,f)

In [28]:
len(correct_incorrect2)

65311